# Bibliotécas

In [1]:
#!pip install awswrangler

In [2]:
!pip install sweetviz

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import awswrangler as wr #Importando um driver para acessar outras instâncias da AWS
import sweetviz as sv
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


# Funções

In [4]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)

In [5]:
#Criando uma função para extrair os dados do Athena 
#def consultar_tabela_athena(database, tabela):
#    wr.config.aws_profile = 'default'
#    wr.config.region = 'us-east-1'
    
#    query = f'SELECT * FROM {database}.{tabela}'
#    df = wr.athena.read_sql_query(query, database = database)    
#    return df

def descricao_estatistica_variaveis_numericas(df):
    #Métricas de tendência central: média e mediana
    central_1 = pd.DataFrame(df.apply(np.mean)).T
    central_2 = pd.DataFrame(df.apply(np.median)).T
    
    #Métricas de dispersão: desvio padrão, mínimo, máximo, aplitude, assimetrica e curtose
    dispersao_1 = pd.DataFrame(df.apply(np.std)).T
    dispersao_2 = pd.DataFrame(df.apply(np.min)).T
    dispersao_3 = pd.DataFrame(df.apply(np.max)).T
    dispersao_4 = pd.DataFrame(df.apply(lambda x: x.max() - x.min())).T
    dispersao_5 = pd.DataFrame(df.apply(lambda x: x.skew())).T
    dispersao_6 = pd.DataFrame(df.apply(lambda x: x.kurtosis())).T
    
    metricas = pd.concat([dispersao_2, dispersao_3, dispersao_4, central_1, central_2, dispersao_1, dispersao_5, dispersao_6]).T
    metricas.columns = ['min', 'max', 'amplitude', 'mean', 'median', 'std', 'assimetria', 'kurtosis']
    return metricas

def descricao_estatistica_variaveis_categoricas(df):
    return df.apply(lambda x: x.unique().shape[0])

In [6]:
#database = 'base_de_dados_catalogados'
#tabela = 'tabela_traintrain'
#df = consultar_tabela_athena(database, tabela)

In [7]:
#df.head(3)

### Salvar os dados com o formato .parquet

In [8]:
# O formato Parquet é altamente eficiente em termos de armazenamento. Ele usa uma codificação eficiente para os dados,
# o que geralmente resulta em arquivos menores em comparação com outros formatos de armazenamento, como CSV ou JSON.

In [9]:
#df.to_parquet('train.parquet', index = False)

In [10]:
#df.shape

### Importar os dados com o formato .parquet

In [11]:
# Obs.: Como os dados estão salvos em memória, não é preciso mais consultar o Athena para trazer os dados

In [12]:
df_parquet = pd.read_parquet('train.parquet')

In [13]:
df_parquet.shape

(1300980, 76)

# 1. (Limpeza dos dados) Passo 1: Descrição dos dados

### 1.1 Descrição das colunas (dicionário de dados)

**SEM_NOT** – Semana Epidemiológica do preenchimento da ficha de notificação.

**SEM_PRI** – Semana Epidemiológica dos Primeiros Sintomas.

**SG_UF_NOT** – Tabela com código e siglas das UF padronizados pelo IBGE.

**(4) ID_REGIONA** – Regional de Saúde de Notificação Código IBGE.
Município realizou a notificação.

**CO_REGIONA** – Regional de Saúde de Notificação Código IBGE.
Município realizou a notificação.

**(5) ID_MUNICIP** – Município Código IBGE.
Unidade que realizou a notificação.

**CO_MUN_NOT** – Município Código IBGE.
Unidade que realizou a notificação.

**(8) CS_SEXO** – Sexo do paciente.

**NU_IDADE_N** – Idade informada pelo paciente.
Quando não se sabe a data de nascimento é registrada a idade aparente. Se digitado a data de nascimento a idade é calculada automaticamente pelo sistema considerando o intervalo entre a data de nascimento e a data dos primeiros sintomas. A idade deve ser <= 150.

**(10) TP_IDADE** – Tipo de idade.
1 – Dia;
2 – Mês;
3 – Ano.
Se digitado a data de nascimento, o campo Idade/Tipo é calculado e preenchido automaticamente pelo sistema: considerando o intervalo entre a data de nascimento e a data dos primeiros sintomas. Se a diferença for de 0 a 30 dias, o sistema grava em Idade = (nº dias) e em Tipo = 1 – Dia. Por exemplo: se Data de nascimento = 05/12/2012 e Data dos 1ºs sintomas = 11/12/2012, então Idade = 6 e Tipo = 1 – Dia. Se a diferença for de 1 a 11 meses, o sistema grava em Idade = (nº meses) e em Tipo = 2 – Mês. Por exemplo: se Data de nascimento = 05/10/2012 e Data dos 1ºs sintomas = 11/12/2012, então Idade = 2 e Tipo = 2 – Mês. Se a diferença for maior ou igual a 12 meses, o sistema grava em Idade = (nº anos) e em Tipo = 3 – Ano. Por exemplo: se Data de nascimento = 05/10/2011 e Data dos 1ºs sintomas = 11/12/2012, então Idade = 1 e Tipo = 3 – Ano.

**COD_IDADE** – Não existe a descrição no dicionário de dados.

**(11) CS_GESTANT** – Gestante Idade gestacional da paciente. 
1 – 1º Trimestre;
2 – 2º Trimestre;
3 – 3º Trimestre;
4 – Idade Gestacional Ignorada;
5 – Não;
6 – Não se aplica;
9 – Ignorado.
Se selecionado categoria 2 – Feminino no campo Sexo. Se selecionado sexo igual a Masculino ou a idade for menor ou igual a 9 anos, o campo é preenchido automaticamente com 6 – Não se aplica. Se selecionado sexo igual a Feminino e idade for maior que 9 anos, o campo não pode ser preenchido com 6 – Não se aplica.

**(12) CS_RACA** – Raça/Cor declarada pelo paciente.
1 – Branca;
2 – Preta;
3 – Amarela;
4 – Parda;
5 – Indígena;
9 – Ignorado.

**(14) CS_ESCOL_N** – Escolaridade.
0 – Sem escolaridade;
1 – Fundamental 1º ciclo (1ª a 5ª série);
2 – Fundamental 2º ciclo (6ª a 9ª série);
3 – Médio (1º ao 3º ano);
4 – Superior;
5 – Não se aplica;
9 – Ignorado.

**(18) SG_UF** – UF.
Unidade federativa de residência do paciente.

**(25) CS_ZONA** – Zona geográfica de residência do paciente.
1 – Urbana;
2 – Rural;
3 – Periurbana;
9 – Ignorado.

**SURTO_SG** – Não está no dicionário.

**(33) NOSOCOMIAL** – Trata–se de caso nosocomial (infecção adquirida no hospital)?.
1 – Sim;
2 – Não;
9 – Ignorado.

**(34) AVE_SUINO** – Paciente trabalha ou tem contato direto com aves, suínos, ou outro animal?.
1 – Sim;
2 – Não;
9 – Ignorado.

**(35) FEBRE** – Sinais e Sintomas/Febre.
1 – Sim;
2 – Não;
9 – Ignorado.

<span style="color:blue">**(35) TOSSE**</span> – Sinais e Sintomas/Tosse.
1 – Sim;
2 – Não;
9 – Ignorado.

**(35) GARGANTA** – Sinais e Sintomas/Dor de Garganta.
1 – Sim;
2 – Não;
9 – Ignorado.

**(35) DISPNEIA** – Sinais e Sintomas/Dispneia (sensação de falta de ar).
1 – Sim;
2 – Não;
9 – Ignorado.

**(35) DESC_RESP** – Sinais e sintomas/desconforto respiratórios.
1 – Sim;
2 – Não;
9 – Ignorado.

**(35) SATURACAO** – Sinais e Sintomas/Saturação O2< 95%.
1 – Sim;
2 – Não;
9 – Ignorado.

**(35) DIARREIA** – Sinais e Sintomas/Diarreia.
1 – Sim;
2 – Não;
9 – Ignorado.

**(35) VÔMITO** – Sinais e Sintomas/Vômito.
1 – Sim;
2 – Não;
9 – Ignorado.

**(35) DOR_ABD** – Sinais e Sintomas/Dor abdominal.
1 – Sim;
2 – Não;
9 – Ignorado.

**(35) FADIGA** – Sinais e Sintomas/Fadiga.
1 – Sim;
2 – Não;
9 – Ignorado.

**(35) ERD_OLFT** – Sinais e Sintomas/Perda do Olfato.
1 – Sim;
2 – Não;
9 – Ignorado.

**(35) PERD_PALA** – Sinais e Sintomas/Perda do Paladar.
1 – Sim;
2 – Não;
9 – Ignorado.

**(35) OUTRO_SIN** – Sinais e Sintomas/Outros.
(String)

**(36) FATOR_RISC** – Fatores de risco (possui algum?).
1 – Sim;
2 – Não;
9 – Ignorado.

**(36) PUERPERA** – Fatores de risco/Puérpera ou parturiente (mulher que pariu recentemente – até 45 dias do parto)?.
1 – Sim;
2 – Não;
9 – Ignorado.

**(36) CARDIOPATI** – Fatores de risco/Cardiovascular Crônica Doença.
1 – Sim;
2 – Não;
9 – Ignorado.

**(36) HEMATOLOGI** – Fatores de risco/Doença hematológica crônica.
1 – Sim;
2 – Não;
9 – Ignorado.

**(36) SIND_DOWN** – Fatores de risco/Síndrome de Down.
1 – Sim;
2 – Não;
9 – Ignorado.

**(36) HEPATICA** – Fatores de risco/Doença Hepática Crônica.
1 – Sim;
2 – Não;
9 – Ignorado.

**(36) ASMA** – Fatores de risco/ Asma.
1 – Sim;
2 – Não;
9 – Ignorado.

**(36) DIABETES** – Fatores de risco/Diabetes mellitus.
1 – Sim;
2 – Não;
9 – Ignorado.

**(36) NEUROLOGIC** – Fatores de risco/doença Neurológica Crônica.
1 – Sim;
2 – Não;
9 – Ignorado.

**(36) PNEUMOPATI** – Fatores de risco/outra Pneumatopatia Crônica (doenças que afetam os pulmões).
1 – Sim;
2 – Não;
9 – Ignorado.

**(36) IMUNODEPRE** – Fatores de risco/Imunodeficiência ou Imunodepressão.
1 – Sim;
2 – Não;
9 – Ignorado.

**(36) RENAL** – Fatores de risco/Doença Renal Crônica.
1 – Sim;
2 – Não;
9 – Ignorado.

**(36) OBESIDADE** – Fatores de risco/Obesidade.
1 – Sim;
2 – Não;
9 – Ignorado.

**(36) OBES_IMC** – Fatores de (Descrição IMC).
Campo habilitado somente se selecionado 1 (Sim) no campo OBESIDADE.

**OUT_MORBI** – Fatores de risco/Outros.
1 – Sim;
2 – Não;
9 – Ignorado.

**MORB_DESC** – Fatores de risco/outros(descrição).
Habilitado se selecionado categoria 1–Sim em Fatores de risco/Outros. OUT_MORBI

**(37) VACINA** – Recebeu vacina contra Gripe na última campanha?.
1 – Sim;
2 – Não;
9 – Ignorado.

**MAE_VAC** – Se < 6 meses: a mãe recebeu a vacina?.
1 – Sim;
2 – Não;
9 – Ignorado.

**M_AMAMENTA** – Se < 6 meses: a mãe amamenta a criança?.
1 – Sim;
2 – Não;
9 – Ignorado.

**(39) ANTIVIRAL** – Usou antiviral para gripe?.
1 – Sim;
2 – Não;
9 – Ignorado.

**(40) TP_ANTIVIR** – Qual antiviral?. Habilitado se campo 39 (ANTIVIRAL) Usou antiviral para gripe? for igual a 1. 
1 – Oseltamivir;
2 – Zanamivir;
3 – Outro, especifique.

**(42) HOSPITAL** – Houve internação?. Caso o campo não seja igual a 1 – Sim o sistema emitirá um aviso indicando que não atende a definição de caso.
1 – Sim;
2 – Não;
9 – Ignorado.

**(47) UTI** – Internado em UTI?.
1 – Sim;
2 – Não;
9 – Ignorado.

**(50) SUPORT_VEN** – Uso de suporte ventilatório?. Suporte ventilatório no intuito de facilitar as trocas gasosas, reduzir a fadiga muscular, reduzir a dispneia, melhorar a ventilação alveolar, aumentar a complacência pulmonar e a capacidade residual funcional, diminuir a morbidade e manter as vias aéreas abertas sem a necessidade de instituir uma via aérea.
1 – Sim, invasivo;
2 – Sim, não invasivo;
3 – Não;
9 – Ignorado.

**(51) RAIOX_RES** – Raio X de Tórax.
1 – Normal;
2 – Infiltrado intersticial;
3 – Consolidação;
4 – Misto;
5 – Outro;
6 – Não realizado;
9 – Ignorado.

**RAIOX_OUT** – Raio X de Tórax/Outro (especificar). Informar o resultado do RX de tórax se selecionado a opção 5–Outro.
Habilitado de campo 57(RAIOX_RES) – Raio X de Tórax = 5 (Outro).

**(53) AMOSTRA** – Coletou amostra. Foi realizado coleta de amostra para realização de teste diagnóstico?
1 – Sim;
2 – Não;
9 – Ignorado.

**(55) TP_AMOSTRA** – Tipo de amostra.
1 – Secreção de Naso–orofaringe;
2 – Lavado Broco–alveolar;
3 – Tecido post–mortem;
4 – Outra, qual?;
5 – LCR;
9 – Ignorado.

**OUT_AMOST** – Tipo de amostra/Outra. Descrição do tipo da amostra clínica, caso diferente das listadas nas categorias do campo.
Campo habilitado se selecionado categoria 4–Outra, qual em Tipo de amostra. em coluna TP_AMOSTRA

<span style="color:red">**CLASSI_FIN**</span> – Classificação final do caso –– TARGET.
1 – SRAG por influenza;
2 – SRAG por outro vírus respiratório;
3 – SRAG por outro agente etiológico qual;
4 – SRAG não especificado;
5 – SRAG por covid–19.

<span style="color:blue">**HISTO_VGM**</span> – Não está no dicionário. Histórico de viagem internacional.

**PAC_COCBO** – Ocupação profissional do paciente.

**PAC_DSCBO** – Ocupação profissional do paciente.

**OUT_ANIM** – Paciente trabalha ou tem contato direto com aves, suínos/Outro animal (especificar). Informar o animal que o paciente teve contato se selecionado a opção 3.
Habilitado de campo 33(AVE_SUINO) – Contato com outro animal = 3 (Outro).

**TOMO_RES** – Aspecto Tomografia.
1 – Tipico covid–19;
2 – Indeterminado covid–19;
3 – Atípico covid–19;
4 – Negativo para Pneumonia;
5 – Outro;
6 – Não realizado;
9 – Ignorado.

**TOMO_OUT** – Aspecto Tomografia/Outro (especificar). Informar o resultado da tomografia se selecionado a opção 5 –Outro
Habilitado de campo 59 (TOMO_RES ) – Aspecto Tomografia = 5

**VACINA_COV** – Recebeu vacina COVID–19?.
1 – Sim;
2 – Não;
9 – Ignorado.

**DOSE_1_COV** – Data 1ª dose da vacina COVID–19.
Habilitado se campo 36(VACINA_COV) – Recebeu vacina COVID–19? for igual a 1.

**DOSE_2_COV** – Data 2ª dose da vacina COVID–19
Habilitado se campo 36(VACINA_COV) – Recebeu vacina COVID–19? for igual a 1.

**DOSE_REF** – Data da dose reforço da vacina COVID–19
Habilitado se campo 36(VACINA_COV) – Recebeu vacina COVID–19? for igual a 1.

**FNT_IN_COV** – Fonte dos dados/informação sobre a vacina COVID–19
Campo preenchido de acordo com a fonte dos dados/informação sobre a vacina COVID–19, se foi digitada manualmente ou recuperada via integração com a Base Nacional de Vacinação.
1 – Manual;
2 – Integração.

**DELTA_UTI** – Não está no dicionário de dados.

**ID** – Não está no dicionário de dados.

### 1.2 Carregar os dados

In [14]:
df1 = df_parquet.copy()

### 1.3 Dimensão dos dados

In [15]:
df1.shape

(1300980, 76)

### 1.4 Editar as colunas

In [16]:
df1.columns

Index(['sem_not', 'sem_pri', 'sg_uf_not', 'id_regiona', 'co_regiona',
       'id_municip', 'co_mun_not', 'cs_sexo', 'nu_idade_n', 'tp_idade',
       'cod_idade', 'cs_gestant', 'cs_raca', 'cs_escol_n', 'sg_uf', 'cs_zona',
       'surto_sg', 'nosocomial', 'ave_suino', 'febre', 'tosse', 'garganta',
       'dispneia', 'desc_resp', 'saturacao', 'diarreia', 'vomito', 'outro_sin',
       'outro_des', 'puerpera', 'fator_risc', 'cardiopati', 'hematologi',
       'sind_down', 'hepatica', 'asma', 'diabetes', 'neurologic', 'pneumopati',
       'imunodepre', 'renal', 'obesidade', 'obes_imc', 'out_morbi',
       'morb_desc', 'vacina', 'mae_vac', 'm_amamenta', 'antiviral',
       'tp_antivir', 'hospital', 'uti', 'suport_ven', 'raiox_res', 'raiox_out',
       'amostra', 'tp_amostra', 'out_amost', 'classi_fin', 'histo_vgm',
       'pac_cocbo', 'pac_dscbo', 'out_anim', 'dor_abd', 'fadiga', 'perd_olft',
       'perd_pala', 'tomo_res', 'tomo_out', 'vacina_cov', 'dose_1_cov',
       'dose_2_cov', 'dose_ref

In [17]:
df1.columns = df1.columns.str.lower()

In [18]:
df1 = df1[['sem_not', 'sem_pri', 'sg_uf_not', 'id_regiona', 'co_regiona',
       'id_municip', 'co_mun_not', 'cs_sexo', 'nu_idade_n', 'tp_idade',
       'cod_idade', 'cs_gestant', 'cs_raca', 'cs_escol_n', 'sg_uf', 'cs_zona',
       'surto_sg', 'nosocomial', 'ave_suino', 'febre', 'tosse', 'garganta',
       'dispneia', 'desc_resp', 'saturacao', 'diarreia', 'vomito', 'dor_abd',
       'fadiga', 'perd_olft', 'perd_pala', 'outro_sin', 'outro_des', 'fator_risc',
       'puerpera', 'cardiopati', 'hematologi', 'sind_down', 'hepatica', 'asma',
       'diabetes', 'neurologic', 'pneumopati', 'imunodepre', 'renal', 'obesidade',
       'obes_imc', 'out_morbi', 'morb_desc', 'vacina', 'mae_vac', 'm_amamenta', 'antiviral',
       'tp_antivir', 'hospital', 'uti', 'suport_ven', 'raiox_res', 'raiox_out',
       'amostra', 'tp_amostra', 'out_amost', 'classi_fin', 'histo_vgm', 'pac_cocbo',
       'pac_dscbo', 'out_anim',  'tomo_res', 'tomo_out', 'vacina_cov', 'dose_1_cov',
       'dose_2_cov', 'dose_ref', 'fnt_in_cov', 'delta_uti', 'id']]

In [19]:
df1_aux = df1.copy()

### 1.5 Selecionar as colunas que inicialmente fazem sentido ao problema abordado

In [20]:
#Eliminando as variáveis que não estão contidas no dicionário ou não tem clareza do que se trata
df1_aux = df1_aux.drop(columns = ['sem_not', 'sem_pri', 'sg_uf_not', 'id_regiona', 'co_regiona', 'id_municip', 'co_mun_not',
                                  'nu_idade_n', 'tp_idade', 'cod_idade', 'obes_imc', 'sg_uf', 'surto_sg', 'outro_sin', 'outro_des',
                                  'out_morbi', 'morb_desc', 'mae_vac', 'm_amamenta', 'raiox_out', 'out_amost', 'histo_vgm', 
                                  'pac_cocbo', 'pac_dscbo', 'out_anim', 'tomo_out', 'fnt_in_cov', 'delta_uti', 'id'], axis = 1)

### 1.6 Limpar inconsistências os dados

In [22]:
df1_aux['cs_sexo'].value_counts()

cs_sexo
M    697225
F    603567
I       188
Name: count, dtype: Int64

In [23]:
# A variável 'cs_sexo' deve conter os dados igual a M ou F. Portanto, os valores diferentes deste foram excluídos

In [24]:
df1_aux = df1_aux[df1_aux['cs_sexo'].isin(['M', 'F'])]

In [25]:
df1_aux['cs_gestant'].value_counts()

cs_gestant
6    843726
5    391893
9     51484
3      8168
2      3297
1      1398
4       671
0       155
Name: count, dtype: Int64

In [26]:
# A variável 'cs_gestant' deve conter os dados 1, 2, 3, 4, 5, 6 e 9. Portanto, os valores diferentes deste foram excluídos

In [27]:
df1_aux = df1_aux[df1_aux['cs_gestant'].isin([1, 2, 3, 4, 5, 6, 9])]

In [28]:
df1_aux['cs_gestant'].value_counts()

cs_gestant
6    843726
5    391893
9     51484
3      8168
2      3297
1      1398
4       671
Name: count, dtype: Int64

In [29]:
df1_aux.loc[df1_aux['cs_sexo'] == 'M', ['cs_gestant']].value_counts()

cs_gestant
6             696033
5               1127
9                 45
Name: count, dtype: int64

In [30]:
# Se a variável 'cs_sexo' é igual a M, então os dados da variável 'cs_gestant' deveriam ser igual a 6 (Não se aplica) 

In [31]:
df1_aux.loc[df1_aux['cs_sexo'] == 'M', 'cs_gestant'] = 6

In [32]:
df1_aux.loc[df1_aux['cs_sexo'] == 'M', 'cs_gestant'].value_counts()

cs_gestant
6    697205
Name: count, dtype: Int64

In [33]:
df1_aux['ave_suino'].value_counts()

ave_suino
2.0    879111
9.0    189169
1.0     11367
3.0      1164
Name: count, dtype: int64

In [34]:
# A variável 'ave_suino' deve conter os dados 1, 2 e 9. Portanto, os valores diferentes deste foram excluídos

In [35]:
df1_aux = df1_aux[df1_aux['ave_suino'].isin([1, 2, 9])]

In [36]:
df1_aux['ave_suino'].value_counts()

ave_suino
2.0    879111
9.0    189169
1.0     11367
Name: count, dtype: int64

In [37]:
df1_aux['puerpera'].value_counts()

puerpera
2.0    372956
9.0      7844
1.0      3606
7.0         1
Name: count, dtype: int64

In [38]:
# A variável 'puerpera' deve conter os dados 1, 2 e 9. Portanto, os valores diferentes deste foram excluídos

In [39]:
df1_aux = df1_aux[df1_aux['puerpera'].isin([1, 2, 9])]

In [40]:
df1_aux['puerpera'].value_counts()

puerpera
2.0    372956
9.0      7844
1.0      3606
Name: count, dtype: int64

In [47]:
df1_aux['tp_antivir'].value_counts()

tp_antivir
2.0    16594
1.0     8652
9.0     2741
3.0     1344
Name: count, dtype: int64

In [49]:
# A variável 'tp_antivir' deve conter os dados 1, 2 e 3. Portanto, os valores diferentes deste foram alterados para 3 (Outro)

In [50]:
df1_aux.loc[df1_aux['tp_antivir'] == 9, 'tp_antivir'] = 3

In [51]:
df1_aux['tp_antivir'].value_counts()

tp_antivir
2.0    16594
1.0     8652
3.0     4085
Name: count, dtype: int64

In [55]:
df1_aux['hospital'].value_counts()

hospital
1.0    350762
2.0      9936
9.0      1066
3.0        87
Name: count, dtype: int64

In [56]:
# A variável 'hospital' deve conter os dados 1, 2 e 9. Portanto, os valores diferentes deste foram alterados para 9 (Outro)

In [57]:
df1_aux.loc[df1_aux['hospital'] == 3, 'hospital'] = 9

In [58]:
df1_aux['hospital'].value_counts()

hospital
1.0    350762
2.0      9936
9.0      1153
Name: count, dtype: int64

In [59]:
df1_aux['uti'].value_counts()

uti
2.0    214642
1.0    142165
9.0      6312
3.0        16
Name: count, dtype: int64

In [60]:
# A variável 'uti' deve conter os dados 1, 2 e 9. Portanto, os valores diferentes deste foram alterados para 9 (Outro)

In [61]:
df1_aux.loc[df1_aux['uti'] == 3, 'uti'] = 9

In [62]:
df1_aux['uti'].value_counts()

uti
2.0    214642
1.0    142165
9.0      6328
Name: count, dtype: int64

In [72]:
df1_aux['amostra'].value_counts()

amostra
1.0           337121
2.0              386
3.0               46
9.0                9
13012022.0         1
13072021.0         1
Name: count, dtype: int64

In [73]:
# A variável 'amostra' deve conter os dados 1, 2, 3, 4, 5 e 9. Portanto, os valores diferentes deste foram excluídos

In [74]:
df1_aux = df1_aux[df1_aux['amostra'].isin([1, 2, 9])]

In [75]:
df1_aux['amostra'].value_counts()

amostra
1.0    337121
2.0       386
9.0         9
Name: count, dtype: int64

In [67]:
df1_aux['tp_amostra'].value_counts()

tp_amostra
1.0           321131
4.0            33767
2.0             1953
9.0              665
6.0              296
5.0              191
3.0              120
5042021.0          1
30052022.0         1
Name: count, dtype: int64

In [68]:
# A variável 'tp_amostra' deve conter os dados 1, 2, 3, 4, 5 e 9. Portanto, os valores diferentes deste foram excluídos

In [69]:
df1_aux = df1_aux[df1_aux['tp_amostra'].isin([1, 2, 3, 4, 5, 9])]

In [70]:
df1_aux['tp_amostra'].value_counts()

tp_amostra
1.0    321131
4.0     33767
2.0      1953
9.0       665
5.0       191
3.0       120
Name: count, dtype: int64

In [77]:
df1_aux['classi_fin'].value_counts()

classi_fin
5    228740
4     98930
1      4078
2      4028
3      1524
6         6
9         2
Name: count, dtype: Int64

In [78]:
# A variável 'classi_fin' deve conter os dados 1, 2, 3, 4 e 5. Portanto, os valores diferentes deste foram excluídos

In [79]:
df1_aux = df1_aux[df1_aux['classi_fin'].isin([1, 2, 3, 4, 5])]

In [80]:
df1_aux['classi_fin'].value_counts()

classi_fin
5    228740
4     98930
1      4078
2      4028
3      1524
Name: count, dtype: Int64

In [84]:
df1_aux['dose_1_cov'].value_counts()

dose_1_cov
0    207164
1    129455
2       203
6       178
9        92
5        58
3        26
4        15
Name: count, dtype: Int64

In [85]:
# A variável 'dose_1_cov' deve conter os dados 0 e 1. Portanto, os valores diferentes deste foram excluídos

In [86]:
df1_aux = df1_aux[df1_aux['dose_1_cov'].isin([0, 1])]

In [87]:
df1_aux['dose_1_cov'].value_counts()

dose_1_cov
0    207164
1    129455
Name: count, dtype: Int64

In [90]:
df1_aux['dose_ref'].value_counts()

dose_ref
0    299907
1     36672
2        20
9        10
Name: count, dtype: Int64

In [91]:
# A variável 'dose_ref' deve conter os dados 0 e 1. Portanto, os valores diferentes deste foram excluídos

In [92]:
df1_aux = df1_aux[df1_aux['dose_ref'].isin([0, 1])]

In [93]:
df1_aux['dose_ref'].value_counts()

dose_ref
0    299907
1     36672
Name: count, dtype: Int64

In [41]:
df1_aux.head(3)

,cs_sexo,cs_gestant,cs_raca,cs_escol_n,cs_zona,nosocomial,ave_suino,febre,tosse,garganta,dispneia,desc_resp,saturacao,diarreia,vomito,dor_abd,fadiga,perd_olft,perd_pala,fator_risc,puerpera,cardiopati,hematologi,sind_down,hepatica,asma,diabetes,neurologic,pneumopati,imunodepre,renal,obesidade,vacina,antiviral,tp_antivir,hospital,uti,suport_ven,raiox_res,amostra,tp_amostra,classi_fin,tomo_res,vacina_cov,dose_1_cov,dose_2_cov,dose_ref
0,M,6,1,9.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,9.0,2.0,NaN,1.0,2.0,3.0,6.0,1.0,1.0,5,6.0,1.0,1,1,0
3,F,5,1,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,NaN,1.0,2.0,2.0,6.0,1.0,1.0,5,6.0,1.0,1,1,1
15,M,6,4,NaN,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,1.0,1.0,1.0,6.0,2.0,NaN,5,1.0,NaN,0,0,0


In [96]:
df1_aux.to_parquet('reduzido.parquet', index = False)

### 1.7 Verificar NA's

In [21]:
df1_NaN = df1_aux.isna().mean().round(4) * 100
sum(df1_NaN)

1493.7299999999998

#### 1.7.1 Tratar NA's

In [ ]:
# Inserindo 9 - não aplicavel nas linhas com NaN
#df1_aux['cs_escol_n'] = df1_aux['cs_escol_n'].fillna(9)
#df1_aux['ave_suino'] = df1_aux['ave_suino'].fillna(9)
#df1_aux['cs_zona'] = df1_aux['cs_zona'].fillna(9)
#df1_aux['nosocomial'] = df1_aux['nosocomial'].fillna(9)
#df1_aux['febre'] = df1_aux['febre'].fillna(9)
#df1_aux['tosse'] = df1_aux['tosse'].fillna(9)
#df1_aux['garganta'] = df1_aux['garganta'].fillna(9)
#df1_aux['dispneia'] = df1_aux['dispneia'].fillna(9)
#df1_aux['desc_resp'] = df1_aux['desc_resp'].fillna(9)
#df1_aux['saturacao'] = df1_aux['saturacao'].fillna(9)
#df1_aux['diarreia'] = df1_aux['diarreia'].fillna(9)
#df1_aux['vomito'] = df1_aux['vomito'].fillna(9)
#df1_aux['outro_sin'] = df1_aux['outro_sin'].fillna(9)
#df1_aux['puerpera'] = df1_aux['puerpera'].fillna(9)
#df1_aux['fator_risc'] = df1_aux['fator_risc'].fillna(9)
#df1_aux['cardiopati'] = df1_aux['cardiopati'].fillna(9)
#df1_aux['hematologi'] = df1_aux['hematologi'].fillna(9)
#df1_aux['sind_down'] = df1_aux['sind_down'].fillna(9)
#df1_aux['hepatica'] = df1_aux['hepatica'].fillna(9)
#df1_aux['asma'] = df1_aux['asma'].fillna(9)
#df1_aux['diabetes'] = df1_aux['diabetes'].fillna(9)
#df1_aux['neurologic'] = df1_aux['neurologic'].fillna(9)
#df1_aux['pneumopati'] = df1_aux['pneumopati'].fillna(9)
#df1_aux['imunodepre'] = df1_aux['imunodepre'].fillna(9)
#df1_aux['renal'] = df1_aux['renal'].fillna(9)
#df1_aux['obesidade'] = df1_aux['obesidade'].fillna(9)
#df1_aux['out_morbi'] = df1_aux['out_morbi'].fillna(9)
#df1_aux['vacina'] = df1_aux['vacina'].fillna(9)
#df1_aux['mae_vac'] = df1_aux['mae_vac'].fillna(9)
#df1_aux['m_amamenta'] = df1_aux['m_amamenta'].fillna(9)
#df1_aux['antiviral'] = df1_aux['antiviral'].fillna(9)
#df1_aux['hospital'] = df1_aux['hospital'].fillna(9)
#df1_aux['uti'] = df1_aux['uti'].fillna(9)
#df1_aux['suport_ven'] = df1_aux['suport_ven'].fillna(9)
#df1_aux['raiox_res'] = df1_aux['raiox_res'].fillna(9)
#df1_aux['tp_amostra'] = df1_aux['tp_amostra'].fillna(9)
#df1_aux['dor_abd'] = df1_aux['dor_abd'].fillna(9)
#df1_aux['perd_pala'] = df1_aux['perd_pala'].fillna(9)
#df1_aux['tomo_res'] = df1_aux['tomo_res'].fillna(9)

### 1.7 Tipo de dados

In [ ]:
#df1_aux.info()

In [ ]:
#Substituindo os vários valores de cada variável pelos valores específicados no dicionários de variáveis
#df1_aux = df1_aux[ ~ df1_aux['ave_suino'].isin([3])]
#df1_aux = df1_aux[df1_aux['vacina_cov'].isin([1, 2, 9])]
#df1_aux = df1_aux[df1_aux['tp_antivir'].isin([1, 2, 3])]
#df1_aux = df1_aux[df1_aux['amostra'].isin([1, 2, 9])]
#df1_aux = df1_aux[df1_aux['tp_amostra'].isin([1, 2, 3, 4, 5, 9])]
#df1_aux = df1_aux[df1_aux['fadiga'].isin([1, 2, 9])]
#df1_aux = df1_aux[df1_aux['perd_olft'].isin([1, 2, 9])]
#df1_aux = df1_aux[df1_aux['dose_1_cov'].isin([0, 1])]
#df1_aux = df1_aux[df1_aux['dose_2_cov'].isin([0, 1])]
#df1_aux = df1_aux[df1_aux['dose_ref'].isin([0, 1])]

#### 1.7.1 Alterado tipo de dados

In [ ]:
# Lista de colunas do tipo float64 que deseja converter para int64
#colunas_float_para_converter = [
#    'cs_escol_n', 'cs_zona', 'nosocomial', 'ave_suino',
#    'febre', 'tosse', 'garganta', 'dispneia', 'desc_resp', 'saturacao',
#    'diarreia', 'vomito', 'outro_sin', 'puerpera', 'cardiopati',
#    'hematologi', 'sind_down', 'hepatica', 'asma', 'diabetes',
#    'neurologic', 'pneumopati', 'imunodepre', 'renal', 'obesidade',
#    'out_morbi', 'vacina', 'mae_vac', 'm_amamenta', 'antiviral',
#    'tp_antivir', 'hospital', 'uti', 'suport_ven', 'raiox_res',
#    'amostra', 'tp_amostra', 'classi_fin', 'dor_abd', 'fadiga',
#    'perd_olft', 'perd_pala', 'tomo_res', 'vacina_cov']

# Convertendo as colunas float64 para int64
#for coluna in colunas_float_para_converter:
    #df1_aux[coluna] = df1_aux[coluna].astype(int)

In [ ]:
#df1_aux['classi_fin'].value_counts().sort_index(ascending = True)

#### 1.6.2 Verificar NA's novamente

In [ ]:
#df1_NaN = df1_aux.isna().mean().round(4) * 100
#sum(df1_NaN)

In [ ]:
#df1_aux.to_parquet('reduzido.parquet', index = False)

In [ ]:
df1_aux = pd.read_parquet('reduzido.parquet')

In [ ]:
df1_aux.head(3)

### <s>1.5 Selecionar amostras aleatórias</s>

In [ ]:
# Devido o data set ter muitas linhas e as classes da variável resposta estar desbalanceadas,
# optou-se fazer uma amostragem com a mesma a quantidade de dados para cada classe.

In [ ]:
#df1_aux.head(3)

In [ ]:
#class1 = df1_aux[df1_aux['classi_fin'] == 1].sample(20000)
#class2 = df1_aux[df1_aux['classi_fin'] == 2].sample(20000)
#class3 = df1_aux[df1_aux['classi_fin'] == 3]
#class4 = df1_aux[df1_aux['classi_fin'] == 4].sample(20000)
#class5 = df1_aux[df1_aux['classi_fin'] == 5].sample(20000)

In [ ]:
#df_classificado = pd.concat([class1, class2, class3, class4, class5], ignore_index = True)

In [ ]:
#df_classificado['classi_fin'].value_counts().sort_index(ascending = True)

### 1.8 Descrição estatística do negócio

#### 1.8.1 Separar as colunas em variáveis numéricas e variáveis categóricas

#### 1.8.2 Atributos numéricos

In [ ]:
atributos_numericos = df1_aux.select_dtypes(include = ['int64'])

In [ ]:
descricao_estatistica_variaveis_numericas(atributos_numericos)

#### 1.8.3 Atributos categóricos

In [ ]:
atributos_categoricos = df1_aux.select_dtypes(exclude = ['int64'])

In [ ]:
descricao_estatistica_variaveis_categoricas(atributos_categoricos)

In [ ]:
correlation = atributos_numericos.corr()['classi_fin']
correlation.sort_values(ascending = False)

In [ ]:
plt.figure(figsize = (30, 20))

correlation = atributos_numericos.corr(method = 'pearson')
mask = np.triu(np.ones_like(correlation))
cmap = sns.color_palette('RdBu_r')
sns.heatmap(correlation, xticklabels = correlation.columns, yticklabels = correlation.columns, cmap = cmap, mask = mask, annot = True);

### 1.9 Selecionar as melhores variáveis

In [ ]:
colunas_selecionadas = df1_aux[['classi_fin', 'nu_idade_n', 'tp_antivir', 'tp_idade', 'sem_pri', 'sem_not']]

In [ ]:
colunas_selecionadas.sample(3)

#### 1.9.1 Visualizar os dados com Sweetviz

In [ ]:
#my_report = sv.analyze(df1_aux)
#my_report.show_html()

In [ ]:
#No Sagemaker, a variável resposta precisa começar com o valor zero (0)
#e também as informações da variável resposta precisa estar na primeira coluna do dataframe, tanto para os dados de treino como os dados de teste  

In [ ]:
colunas_selecionadas['classi_fin'].value_counts()

In [ ]:
# Criando um dicionário de mapeamento
mapeamento = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4}

colunas_selecionadas['classi_fin'] = colunas_selecionadas['classi_fin'].map(mapeamento)

In [ ]:
colunas_selecionadas['classi_fin'].value_counts()

# EDA

**Pessoas que não tomaram vacina tem maior nº de casos de covid?

**Pessoas que estão em área urbana a tendencia de contrair covid é maior?

**Pessoas que tem contato com animais do tipo ave ou suino tem alguma doença respiratória?

**Pessoas com maior idade tem maior tendencia a contrair doença respiratória?

**Pessoas que vivem no interior de Santa Catarina não tiveram nenhuma doença respiratória?

**Pessoas que fumam tem tendencia a alguma doença respiratória?

**Semanas do ano que são mais frias a quantidade de doença respiratória é maior?

**Pessoas que tomaram antiviral tiveram menos sintomas graves?

### 2. Criar um novo bucket no S3 para treinar o algoritmo do Sagemaker

In [ ]:
df2 = colunas_selecionadas.copy()

In [ ]:
#df2 = df1_aux.copy()

#### 2.1 Define o nome do bucket S3 para armazenar os dados e os modelos

In [ ]:
#Salvando em memória as informações do bucket criado
bucket = 'leonardo-nassib-fonseca-treinamento-modelo-do-sagemaker' #Nome do bucket no S3s3://leonardo-nassib-fonseca-treinamento-modelo-do-sagemaker/treino/
prefix1 = 'treino' #Nome da pasta onde chamado treino
prefix2 = 'teste' #Nome da pasta onde chamado teste
prefixest = 'est'

In [ ]:
#Separando os dados em treino e teste
x = df2.drop('classi_fin', axis = 1)
y = df2['classi_fin']
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
x_treino.shape, x_teste.shape, y_treino.shape, y_teste.shape

#### 2.2 Salvar os dados em memória

In [ ]:
dados_treino = pd.concat([y_treino, x_treino], axis = 1)
dados_teste = pd.concat([y_teste, x_teste], axis = 1)

In [ ]:
#No Sagemaker é necessário remover o nome das colunas

In [ ]:
#Dados salvos em memória
dados_treino.to_csv('treino.csv', index = False, header = False)
dados_teste.to_csv('teste.csv', index = False, header = False)

#### 2.3 Enviar os dados para o S3

In [ ]:
# Configurar a sessão do SageMaker, seria uma espécie de login no SageMaker
sagemaker_session = sagemaker.Session()

In [ ]:
# Fazendo upload das pastas e dos arquivos de treinamento e teste para o bucket S3
dados_treino_path = sagemaker_session.upload_data(path = 'treino.csv', bucket = bucket, key_prefix = prefix1)
dados_teste_path = sagemaker_session.upload_data(path = 'teste.csv', bucket = bucket, key_prefix = prefix2)

In [ ]:
#Esses objetos TrainingInput são usados posteriormente no Amazon SageMaker
#para configurar e iniciar o treinamento do modelo de machine learning.
#Eles especificam onde os dados estão armazenados no Amazon S3 e o formato
#dos dados. Essa abordagem permite que o SageMaker acesse facilmente
#os dados necessários para treinar e testar o modelo.
s3_input_treino = sagemaker.inputs.TrainingInput(s3_data = 's3://{}/{}/'.format(bucket, prefix1), content_type = 'csv')
s3_input_teste = sagemaker.inputs.TrainingInput(s3_data = 's3://{}/{}/'.format(bucket, prefix2), content_type = 'csv')

#### 2.4 Configurar o ambiente de treinamento do Sagemaker

In [ ]:
#Essas linhas de código configuram um ambiente de treinamento no Amazon SageMaker
#usando o algoritmo XGBoost, especificando a função de execução, o tipo e o número de
#instâncias de treinamento, e o local para armazenamento dos resultados do treinamento.
regras = get_execution_role()

container = get_image_uri(sagemaker_session.boto_region_name, 'xgboost')

estimator = sagemaker.estimator.Estimator(container,
                                          regras,
                                          train_instance_count = 1,
                                          train_instance_type = 'ml.m4.xlarge', #Tipo de máquina
                                          output_path = 's3://{}/{}/output'.format(bucket, prefixest), #Onde será salvo a saida do algoritmo
                                          sagemaker_session = sagemaker_session)

In [ ]:
regras #Qual regra está associada 

In [ ]:
container #Nome do container

In [ ]:
estimator

#### 2.5 Definir os hiperparâmetros do modelo

In [ ]:
# Definir hiperparâmetros para o modelo XGBoost
estimator.set_hyperparameters(objective = 'multi:softmax', #multi:softmax para modelos multiclasses, para modelos binarios seria outro parâmetro
                              num_class = 5, #Quantidade de classes
                              num_round = 100)

#### 2.6 Treinar o modelo

In [ ]:
# Treinando o modelo
estimator.fit({'train': s3_input_treino, 'validation': s3_input_teste})

In [ ]:
# Implantando o modelo treinado em um endpoint no Amazon SageMaker. Este modelo treinado fica em Inferência > EndPoints
predictor = estimator.deploy(initial_instance_count = 1, instance_type='ml.m4.xlarge')